In [1]:
import pandas as pd
import os.path
import pickle
import numpy as np
import keras.utils
import time
from keras.callbacks import TensorBoard, CSVLogger
# from keras.preprocessing.text import text_to_word_sequence
# from keras.preprocessing import sequence
# from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text import text_to_word_sequence, Tokenizer
from tensorflow.keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense,Flatten,LSTM,Conv1D,GlobalMaxPool1D,Dropout,Bidirectional
# from keras.layers.embeddings import Embedding
from tensorflow.keras.layers import Embedding
from keras import optimizers
from keras.layers import Input
from keras.models import Model
# from keras.utils.vis_utils import plot_model
from IPython.display import SVG
# from keras.utils.vis_utils import model_to_dot
from tensorflow.keras.utils import plot_model, model_to_dot
from keras.models import load_model
from nltk.corpus import stopwords
import operator

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tamra\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import spacy
# !python -m spacy download en
# nlp = spacy.load('en')

nlp = spacy.load('en_core_web_sm')


In [ ]:

pd.options.display.max_rows
# pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_colwidth', None)
# from keras import backend as K
# K.tensorflow_backend._get_available_gpus()
from tensorflow.config import list_physical_devices
list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [7]:
!pip install pydot==1.2.3

In [9]:
train_data = pd.read_table('train.tsv', names = ["id", "label", "statement", "subject", "speaker", "job", "state", "party",
                                            "barely-true", "false", "half-true", "mostly-true", "pants-fire", "venue"])

val_data = pd.read_table('valid.tsv', names = ["id", "label", "statement", "subject", "speaker", "job", "state", "party",
                                            "barely-true", "false", "half-true", "mostly-true", "pants-fire", "venue"])

test_data = pd.read_table('test.tsv', names = ["id", "label", "statement", "subject", "speaker", "job", "state", "party",
                                            "barely-true", "false", "half-true", "mostly-true", "pants-fire", "venue"])


In [11]:
print(train_data.shape, val_data.shape, test_data.shape)
print(train_data.label.unique())


(10240, 14) (1284, 14) (1267, 14)
['false' 'half-true' 'mostly-true' 'true' 'barely-true' 'pants-fire']


In [13]:
print(train_data.info())
print(val_data.info())
print(test_data.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10240 entries, 0 to 10239
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           10240 non-null  object 
 1   label        10240 non-null  object 
 2   statement    10240 non-null  object 
 3   subject      10238 non-null  object 
 4   speaker      10238 non-null  object 
 5   job          7342 non-null   object 
 6   state        8030 non-null   object 
 7   party        10238 non-null  object 
 8   barely-true  10238 non-null  float64
 9   false        10238 non-null  float64
 10  half-true    10238 non-null  float64
 11  mostly-true  10238 non-null  float64
 12  pants-fire   10238 non-null  float64
 13  venue        10138 non-null  object 
dtypes: float64(5), object(9)
memory usage: 1.1+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1284 entries, 0 to 1283
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype 
---  ------   

In [15]:
#####################
### OUTPUT LABELS ###
#####################

y_label_dict = {
    "pants-fire": 0,
    "false": 1,
    "barely-true": 2,
    "half-true": 3,
    "mostly-true": 4,
    "true": 5
}

print(y_label_dict)

train_data['output'] = train_data['label'].apply(lambda x: y_label_dict[x])
val_data['output'] = val_data['label'].apply(lambda x: y_label_dict[x])
test_data['output'] = test_data['label'].apply(lambda x: y_label_dict[x])

num_classes = 6


{'pants-fire': 0, 'false': 1, 'barely-true': 2, 'half-true': 3, 'mostly-true': 4, 'true': 5}


In [17]:
#####################
###   SPEAKERS    ###
#####################

frequent_speakers = {
    'barack-obama': 0, 'donald-trump': 1, 'hillary-clinton': 2,
    'mitt-romney': 3, 'scott-walker': 4, 'john-mccain': 5,
    'rick-perry': 6, 'chain-email': 7, 'marco-rubio': 8, 'viral-image': 13,
    'rick-scott': 9, 'ted-cruz': 10, 'bernie-s': 11, 'newt-gingrich': 16,
    'chris-christie': 12, 'facebook-posts': 13, 'blog-posting': 13,
    'charlie-crist': 14, 'congressional': 15, 'republican': 15,
    'national-committe': 15, 'democratic': 15
}

print(frequent_speakers)

def get_speaker_id(speaker):
    if isinstance(speaker, str):
        matched = [sp for sp in frequent_speakers if sp in speaker.lower()]
        if len(matched) > 0:
            return frequent_speakers[matched[0]]
        else:
            return len(set(frequent_speakers.values()))
    else:
        return len(set(frequent_speakers.values()))

train_data['speaker_id'] = train_data['speaker'].apply(get_speaker_id)
val_data['speaker_id'] = val_data['speaker'].apply(get_speaker_id)
test_data['speaker_id'] = test_data['speaker'].apply(get_speaker_id)

print(len(set(frequent_speakers.values())))

train_data['speaker_id'].value_counts()


{'barack-obama': 0, 'donald-trump': 1, 'hillary-clinton': 2, 'mitt-romney': 3, 'scott-walker': 4, 'john-mccain': 5, 'rick-perry': 6, 'chain-email': 7, 'marco-rubio': 8, 'viral-image': 13, 'rick-scott': 9, 'ted-cruz': 10, 'bernie-s': 11, 'newt-gingrich': 16, 'chris-christie': 12, 'facebook-posts': 13, 'blog-posting': 13, 'charlie-crist': 14, 'congressional': 15, 'republican': 15, 'national-committe': 15, 'democratic': 15}
17


,count
speaker_id,
17,7348
0,488
15,347
1,275
2,239
3,176
13,156
4,149
5,148


In [19]:
#####################
###   JOB TITLE   ###
#####################

frequent_jobs = {
    'senator': 0, 'president': 1, 'governor': 2,
    'u.s. representative': 3, 'attorney': 4, 'congressman': 5,
    'congresswoman': 5, 'social media posting': 6, 'lawyer': 4,
    'businessman': 6, 'radio host': 8, 'host': 8,
    'mayor': 7, 'assembly': 9, 'representative': 3,
    'senate': 9, 'state representative': 10, 'milwaukee county executive': 11,
    'u.s. house of representatives': 3, 'house representative': 3,
    'house of representatives': 3, 'house member': 3
}

print(frequent_jobs)

def get_job_id(job):
    if isinstance(job, str):
        matched = [jb for jb in frequent_jobs if jb in job.lower()]
        if len(matched) > 0:
            return frequent_jobs[matched[0]]
        else:
            return len(set(frequent_jobs.values()))
    else:
        return len(set(frequent_jobs.values()))

train_data['job_id'] = train_data['job'].apply(get_job_id)
val_data['job_id'] = val_data['job'].apply(get_job_id)
test_data['job_id'] = test_data['job'].apply(get_job_id)

print(len(set(frequent_jobs.values())))

train_data['job_id'].value_counts()


{'senator': 0, 'president': 1, 'governor': 2, 'u.s. representative': 3, 'attorney': 4, 'congressman': 5, 'congresswoman': 5, 'social media posting': 6, 'lawyer': 4, 'businessman': 6, 'radio host': 8, 'host': 8, 'mayor': 7, 'assembly': 9, 'representative': 3, 'senate': 9, 'state representative': 10, 'milwaukee county executive': 11, 'u.s. house of representatives': 3, 'house representative': 3, 'house of representatives': 3, 'house member': 3}
12


,count
job_id,
12,4597
1,1212
0,1201
3,911
2,892
8,279
9,253
5,232
4,223


In [25]:
#####################
###     PARTY     ###
#####################

# Generate the most frequent parties and reset the index
party_counts = train_data['party'].str.lower().value_counts()[:5].reset_index()

# Print the column names of the party_counts DataFrame to inspect
print(party_counts.columns)

# Assuming the first column after reset_index() is the party name
# Extract party names from the first column (the column after reset_index)
frequent_parties = dict(zip(party_counts.iloc[:, 0], range(len(party_counts))))

print(frequent_parties)

# Function to get the party ID based on party name
def get_party_id(party):
    if isinstance(party, str):
        matched = [pt for pt in frequent_parties if pt in party.lower()]
        if len(matched) > 0:
            return frequent_parties[matched[0]]
        else:
            return len(set(frequent_parties.values()))
    else:
        return len(set(frequent_parties.values()))

# Apply the function to train, validation, and test data
train_data['party_id'] = train_data['party'].apply(get_party_id)
val_data['party_id'] = val_data['party'].apply(get_party_id)
test_data['party_id'] = test_data['party'].apply(get_party_id)

# Print the number of unique party IDs
print(len(set(frequent_parties.values())))

# Check the frequency of each party ID in the train data
train_data['party_id'].value_counts()


Index(['party', 'count'], dtype='object')
{'republican': 0, 'democrat': 1, 'none': 2, 'organization': 3, 'independent': 4}
5


,count
party_id,
0,4497
1,3337
2,1744
5,296
3,219
4,147


In [26]:
train_data.loc[train_data['party_id']==9]['party'].value_counts()[:90]

,count
party,


In [28]:
#####################
###     STATE     ###
#####################

# Define other states (You can uncomment this part if needed)
# states_50 = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', ...]

# List of states categorized as "other states"
other_states = ['wyoming', 'colorado', 'hawaii', 'tennessee', 'nevada', 'maine',
                'north dakota', 'mississippi', 'south dakota', 'oklahoma',
                'delaware', 'minnesota', 'north carolina', 'arkansas', 'indiana',
                'maryland', 'louisiana', 'idaho', 'iowa', 'west virginia',
                'michigan', 'kansas', 'utah', 'connecticut', 'montana', 'vermont',
                'pennsylvania', 'alaska', 'kentucky', 'nebraska', 'new hampshire',
                'missouri', 'south carolina', 'alabama', 'new mexico']

# Define frequent states
frequent_states = {'texas': 1, 'florida': 2, 'wisconsin': 3, 'new york': 4,
                    'illinois': 5, 'ohio': 6, 'georgia': 7, 'virginia': 8,
                   'rhode island': 9, 'oregon': 10, 'new jersey': 11,
                   'massachusetts': 12, 'arizona': 13, 'california': 14,
                   'washington': 15}

# Assign ID 0 for other states
for state in other_states:
    frequent_states[state] = 0

# Print frequent states dictionary
print(frequent_states)

# Function to get the state ID based on state name
def get_state_id(state):
    if isinstance(state, str):
        if state.lower().rstrip() in frequent_states:
            return frequent_states[state.lower().rstrip()]
        else:
            if 'washington' in state.lower():
                return frequent_states['washington']
            else:
                return len(set(frequent_states.values()))
    else:
        return len(set(frequent_states.values()))

# Apply the function to train, validation, and test data
train_data['state_id'] = train_data['state'].apply(get_state_id)
val_data['state_id'] = val_data['state'].apply(get_state_id)
test_data['state_id'] = test_data['state'].apply(get_state_id)

# Print the number of unique state IDs
print(len(set(frequent_states.values())))

# Check the frequency of each state ID in the train data
train_data['state_id'].value_counts()


{'texas': 1, 'florida': 2, 'wisconsin': 3, 'new york': 4, 'illinois': 5, 'ohio': 6, 'georgia': 7, 'virginia': 8, 'rhode island': 9, 'oregon': 10, 'new jersey': 11, 'massachusetts': 12, 'arizona': 13, 'california': 14, 'washington': 15, 'wyoming': 0, 'colorado': 0, 'hawaii': 0, 'tennessee': 0, 'nevada': 0, 'maine': 0, 'north dakota': 0, 'mississippi': 0, 'south dakota': 0, 'oklahoma': 0, 'delaware': 0, 'minnesota': 0, 'north carolina': 0, 'arkansas': 0, 'indiana': 0, 'maryland': 0, 'louisiana': 0, 'idaho': 0, 'iowa': 0, 'west virginia': 0, 'michigan': 0, 'kansas': 0, 'utah': 0, 'connecticut': 0, 'montana': 0, 'vermont': 0, 'pennsylvania': 0, 'alaska': 0, 'kentucky': 0, 'nebraska': 0, 'new hampshire': 0, 'missouri': 0, 'south carolina': 0, 'alabama': 0, 'new mexico': 0}
16


,count
state_id,
16,2238
0,1211
1,1009
2,1003
3,714
4,659
5,558
6,448
7,433


In [30]:
#####################
###    SUBJECT    ###
#####################

# Define frequent subjects
frequent_subjects = {'health': 0, 'tax': 1, 'immigration': 2, 'election': 3,
                     'education': 4, 'candidates-biography': 5, 'economy': 6,
                     'gun': 7, 'job': 8, 'federal-budget': 6, 'energy': 9,
                     'abortion': 10, 'foreign-policy': 6, 'state-budget': 6,
                     'crime': 11, 'gays-and-lesbians' : 12, 'medicare' : 0,
                     'terrorism' : 11, 'finance' : 6, 'criminal': 11,
                     'transportation': 13}

# Print frequent_subjects dictionary
print(frequent_subjects)

# Function to get the subject ID based on the subject name
def get_subject_id(subject):
    if isinstance(subject, str):
        matched = [sbj for sbj in frequent_subjects if sbj in subject.lower()]
        if len(matched) > 0:
            return frequent_subjects[matched[0]]
        else:
            return len(set(frequent_subjects.values()))
    else:
        return len(set(frequent_subjects.values()))

# Apply the function to train, validation, and test data
train_data['subject_id'] = train_data['subject'].apply(get_subject_id)
val_data['subject_id'] = val_data['subject'].apply(get_subject_id)
test_data['subject_id'] = test_data['subject'].apply(get_subject_id)

# Print the number of unique subject IDs
print(len(set(frequent_subjects.values())))

# Check the frequency of each subject ID in the train data
train_data['subject_id'].value_counts()

#print(train_data.loc[train_data['subject_id']==15]['subject'].value_counts())


{'health': 0, 'tax': 1, 'immigration': 2, 'election': 3, 'education': 4, 'candidates-biography': 5, 'economy': 6, 'gun': 7, 'job': 8, 'federal-budget': 6, 'energy': 9, 'abortion': 10, 'foreign-policy': 6, 'state-budget': 6, 'crime': 11, 'gays-and-lesbians': 12, 'medicare': 0, 'terrorism': 11, 'finance': 6, 'criminal': 11, 'transportation': 13}
14


,count
subject_id,
6,2103
14,1909
0,1302
1,906
4,621
3,569
5,512
2,506
11,438


In [32]:
#####################
###     VENUE     ###
#####################

# Define frequent venues with their IDs
frequent_venues = {'news release' : 0, 'interview' : 1, 'press release' : 2,
                   'speech' : 3, 'tv' : 4, 'tweet' : 5, 'campaign' : 6,
                   'television' : 4, 'debate' : 7, 'news conference' : 8,
                   'facebook' : 5, 'press conference' : 8, 'radio' : 9,
                   'e-mail' : 10, 'email' : 10, 'mail' : 10, 'social media' : 5,
                   'twitter' : 5, 'blog': 11, 'article': 11, 'comment': 12, 'web': 11}

# Print frequent_venues dictionary
print(frequent_venues)

# Function to get the venue ID based on venue name
def get_venue_id(venue):
    if isinstance(venue, str):
        matched = [ven for ven in frequent_venues if ven in venue.lower()]
        if len(matched) > 0:
            return frequent_venues[matched[0]]
        else:
            return len(set(frequent_venues.values()))
    else:
        return len(set(frequent_venues.values()))

# Apply the function to train, validation, and test data
train_data['venue_id'] = train_data['venue'].apply(get_venue_id)
val_data['venue_id'] = val_data['venue'].apply(get_venue_id)
test_data['venue_id'] = test_data['venue'].apply(get_venue_id)

# Print the number of unique venue IDs
print(len(set(frequent_venues.values())))

# Check the frequency of each venue ID in the train data
train_data['venue_id'].value_counts()

#print(train_data.loc[train_data['venue_id']==13]['venue'].value_counts())


{'news release': 0, 'interview': 1, 'press release': 2, 'speech': 3, 'tv': 4, 'tweet': 5, 'campaign': 6, 'television': 4, 'debate': 7, 'news conference': 8, 'facebook': 5, 'press conference': 8, 'radio': 9, 'e-mail': 10, 'email': 10, 'mail': 10, 'social media': 5, 'twitter': 5, 'blog': 11, 'article': 11, 'comment': 12, 'web': 11}
13


,count
venue_id,
13,2695
1,1752
3,1059
7,735
6,679
4,568
11,529
5,473
10,356


In [33]:
train_data.head()

,id,label,statement,subject,speaker,job,state,party,barely-true,false,...,mostly-true,pants-fire,venue,output,speaker_id,job_id,party_id,state_id,subject_id,venue_id
0,2635.json,false,Says the Annies List political group supports third-trimester abortions on demand.,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,...,0.0,0.0,a mailer,1,17,3,0,1,10,10
1,10540.json,half-true,When did the decline of coal start? It started when natural gas took off that started to begin in (President George W.) Bushs administration.,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,...,1.0,0.0,a floor speech.,3,17,12,1,8,8,3
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by voting to give George Bush the benefit of the doubt on Iran.""",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,...,163.0,9.0,Denver,4,0,1,1,5,6,13
3,1123.json,false,Health care reform legislation is likely to mandate free sex change surgeries.,health-care,blog-posting,NaN,NaN,none,7.0,19.0,...,5.0,44.0,a news release,1,13,12,2,16,0,0
4,9028.json,half-true,The economic turnaround started at the end of my term.,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,...,19.0,2.0,an interview on CNN,3,14,12,1,2,6,1


In [39]:
#####################
###   STATEMENT   ###
#####################

import os
import pickle
from tensorflow.keras.preprocessing.text import text_to_word_sequence, Tokenizer
from nltk.corpus import stopwords

def load_statement_vocab_dict(train_data):
    vocabulary_dict = {}
    if not os.path.exists('vocabulary.p'):
        tokenizer = Tokenizer()
        tokenizer.fit_on_texts(train_data['statement'])
        vocabulary_dict = tokenizer.word_index
        print(len(vocabulary_dict))  # Corrected print statement
        pickle.dump(vocabulary_dict, open("vocabulary.p", "wb"))  # Use pickle instead of cPickle
        print('Created Vocabulary Dictionary...')
        print('Saved Vocabulary Dictionary...')
    else:
        print('Loading Vocabulary Dictionary...')
        vocabulary_dict = pickle.load(open("vocabulary.p", "rb"))
    return vocabulary_dict


def preprocess_statement(statement):
    statement = [w for w in statement.split(' ') if w not in stopwords.words('english')]
    statement = ' '.join(statement)
    text = text_to_word_sequence(statement)
    val = [0] * 10
    val = [vocabulary_dict[t] for t in text if t in vocabulary_dict]
    return val


# Load vocabulary dictionary
vocabulary_dict = load_statement_vocab_dict(train_data)

# Apply the preprocessing function to train, validation, and test data
train_data['word_id'] = train_data['statement'].apply(preprocess_statement)
val_data['word_id'] = val_data['statement'].apply(preprocess_statement)
test_data['word_id'] = test_data['statement'].apply(preprocess_statement)


12408
Created Vocabulary Dictionary...
Saved Vocabulary Dictionary...


In [41]:
#####################
###   POS TAGS    ###
#####################

pos_tags = {'ADJ': 'adjective', 'ADP': 'adposition', 'ADV': 'adverb',
            'AUX': 'auxiliary verb', 'CONJ': 'coordinating conjunction',
            'DET': 'determiner', 'INTJ': 'interjection', 'NOUN': 'noun',
            'NUM': 'numeral', 'PART': 'particle', 'PRON': 'pronoun',
            'PROPN': 'proper noun', 'PUNCT': 'punctuation', 'X': 'other',
            'SCONJ': 'subord conjunction', 'SYM': 'symbol', 'VERB': 'verb'}

pos_dict = {'NOUN' : 0, 'VERB' : 1, 'ADP' : 2, 'PROPN' : 3, 'PUNCT' : 4,
            'DET' : 5, 'ADJ' : 6, 'NUM' : 7, 'ADV' : 8, 'PRON' : 9, 'X' : 9,
            'PART' : 9, 'SYM' : 9, 'INTJ' : 9 }

dep_dict = {'ACL' : 0, 'ACOMP' : 1, 'ADVCL' : 2, 'ADVMOD' : 3, 'AGENT' : 4,
            'AMOD' : 5, 'APPOS' : 6, 'ATTR' : 7, 'AUX' : 8, 'AUXPASS' : 9,
            'CASE' : 10, 'CC' : 11, 'CCOMP' : 12, 'COMPOUND' : 13, 'CONJ' : 14,
            'CSUBJ' : 15, 'CSUBJPASS' : 16, 'DATIVE' : 17, 'DEP' : 18,
            'DET' : 19, 'DOBJ' : 20, 'EXPL' : 21, 'INTJ' : 22, 'MARK' : 23,
            'META' : 24, 'NEG' : 25, 'NOUNMOD' : 26, 'NPMOD' : 27, 'NSUBJ' : 28,
            'NSUBJPASS' : 29, 'NUMMOD' : 30, 'OPRD' : 31, 'PARATAXIS' : 32,
            'PCOMP' : 33, 'POBJ' : 34, 'POSS' : 35, 'PRECONJ' : 36, 'PREDET' : 37,
            'PREP' : 38, 'PRT' : 39, 'PUNCT' : 40, 'QUANTMOD' : 41,
            'RELCL' : 42, 'ROOT' : 43, 'XCOMP' : 44}

def get_pos(statement):
    # Removing .decode() because it's unnecessary in Python 3
    doc = nlp(statement)  # Use the string directly
    taglist = []
    deplist = []
    for token in doc:
        taglist.append(pos_dict.get(token.pos_, max(pos_dict.values())))
        # deplist.append(token.dep_)
    return taglist

train_data['pos_id'] = train_data['statement'].apply(get_pos)
val_data['pos_id'] = val_data['statement'].apply(get_pos)
test_data['pos_id'] = test_data['statement'].apply(get_pos)


In [43]:
#####################
###   DEP PARSE   ###
#####################

dep_dict = {'punct' : 0, 'prep' : 1, 'pobj' : 2, 'compound' : 3, 'det' : 4,
            'nsubj' : 5, 'ROOT' : 6, 'amod' : 7, 'dobj' : 8, 'aux' : 9,
            'advmod' : 10, 'nummod' : 10, 'ccomp' : 10, 'conj' : 10, 'cc' : 10,
            'advcl' : 10, 'poss' : 10, 'mark' : 10, 'quantmod' : 10, 'relcl' : 10,
            'attr' : 10, 'xcomp' : 10, 'npadvmod' : 10, 'nmod' : 10, 'auxpass' : 10,
            'acl' : 10, 'nsubjpass' : 10, 'pcomp' : 10, 'acomp' : 10, 'neg' : 10,
            'appos' : 10, 'prt' : 10, '' : 10, 'expl' : 10, 'dative' : 10,
            'agent' : 10, 'case' : 10, 'oprd' : 10, 'csubj' : 10, 'dep' : 10,
            'intj' : 10, 'predet' : 10, 'parataxis' : 10, 'preconj' : 10,
            'meta' : 10, 'csubjpass' : 10}

def get_dep_parse(statement):
    # Removed .decode() since it's not necessary in Python 3
    doc = nlp(statement)  # Use the string directly
    deplist = []
    for token in doc:
        deplist.append(dep_dict.get(token.dep_, max(dep_dict.values())))
    return deplist

# Apply the dependency parsing function to train, validation, and test data
train_data['dep_id'] = train_data['statement'].apply(get_dep_parse)
val_data['dep_id'] = val_data['statement'].apply(get_dep_parse)
test_data['dep_id'] = test_data['statement'].apply(get_dep_parse)


In [44]:
train_data.head()

,id,label,statement,subject,speaker,job,state,party,barely-true,false,...,output,speaker_id,job_id,party_id,state_id,subject_id,venue_id,word_id,pos_id,dep_id
0,2635.json,false,Says the Annies List political group supports third-trimester abortions on demand.,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,...,1,17,3,0,1,10,10,"[7, 6968, 1141, 520, 621, 385, 444, 5119, 585, 1601]","[1, 5, 3, 3, 6, 0, 1, 6, 4, 0, 0, 2, 0, 4]","[6, 4, 10, 10, 7, 5, 10, 7, 0, 3, 8, 1, 2, 0]"
1,10540.json,half-true,When did the decline of coal start? It started when natural gas took off that started to begin in (President George W.) Bushs administration.,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,...,3,17,12,1,8,8,3,"[63, 2091, 964, 866, 23, 602, 1142, 315, 180, 602, 1959, 34, 310, 560, 1365, 177]","[9, 1, 5, 0, 2, 0, 0, 4, 9, 1, 9, 6, 0, 1, 2, 9, 1, 9, 1, 2, 4, 3, 3, 3, 4, 3, 0, 4]","[10, 6, 4, 5, 1, 3, 2, 0, 5, 6, 10, 7, 5, 10, 10, 5, 10, 9, 10, 1, 0, 3, 3, 10, 0, 3, 8, 0]"
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by voting to give George Bush the benefit of the doubt on Iran.""",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,...,4,0,1,1,5,6,13,"[127, 101, 3546, 191, 254, 20, 329, 343, 310, 166, 1093, 3547, 416]","[3, 3, 1, 2, 3, 3, 4, 2, 1, 9, 1, 3, 3, 5, 0, 2, 5, 0, 2, 3, 4, 4]","[3, 5, 6, 1, 3, 2, 0, 1, 10, 9, 10, 3, 10, 4, 8, 1, 4, 2, 1, 2, 0, 0]"
3,1123.json,false,Health care reform legislation is likely to mandate free sex change surgeries.,health-care,blog-posting,NaN,NaN,none,7.0,19.0,...,1,13,12,2,16,0,0,"[32, 43, 266, 298, 666, 667, 404, 467, 417, 4148]","[0, 0, 0, 0, 9, 6, 9, 1, 6, 0, 0, 0, 4]","[3, 3, 3, 5, 6, 10, 9, 10, 7, 3, 3, 8, 0]"
4,9028.json,half-true,The economic turnaround started at the end of my term.,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,...,3,14,12,1,2,6,1,"[1, 325, 4149, 602, 408, 505]","[5, 6, 0, 1, 2, 5, 0, 2, 9, 0, 4]","[4, 7, 5, 6, 1, 4, 2, 1, 10, 2, 0]"


In [47]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip -d ./glove

--2025-04-15 20:37:17--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2025-04-15 20:37:17--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-04-15 20:37:17--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [50]:
#####################
###  EMBEDDINGS   ###
#####################

import numpy as np  # Just in case it's not imported yet

embeddings = {}
with open("glove/glove.6B.100d.txt", encoding='utf-8') as file_object:
    for line in file_object:
        word_embed = line.split()
        word = word_embed[0]
        embed = np.array(word_embed[1:], dtype="float32")
        embeddings[word.lower()] = embed

EMBED_DIM = 100
print(len(embeddings), " : Word Embeddings Found")
print(len(embeddings[word]), " : Embedding Dimension")

num_words = len(vocabulary_dict) + 1
embedding_matrix = np.zeros((num_words, EMBED_DIM))
for word, i in vocabulary_dict.items():
    embedding_vector = embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Clear unused embedding index
embeddings_index = None

# Create one-hot embeddings for POS and DEP tags
pos_embeddings = np.identity(max(pos_dict.values()) + 1, dtype=int)
dep_embeddings = np.identity(max(dep_dict.values()) + 1, dtype=int)


400000  : Word Embeddings Found
100  : Embedding Dimension


In [52]:
#####################
###  HYPERPARAMS  ###
#####################
vocab_length = len(vocabulary_dict.keys())
hidden_size = EMBED_DIM  # Has to be same as EMBED_DIM
lstm_size = 100
num_steps = 15
num_epochs = 30
batch_size = 40

# Hyperparams for CNN
kernel_sizes = [3, 3, 3]
filter_size = 128

# Meta data related hyper params
num_party = len(train_data.party_id.unique())
num_state = len(train_data.state_id.unique())
num_venue = len(train_data.venue_id.unique())
num_job = len(train_data.job_id.unique())
num_sub = len(train_data.subject_id.unique())
num_speaker = len(train_data.speaker_id.unique())

print(num_party)
print(num_state)
print(num_venue)
print(num_job)
print(num_sub)
print(num_speaker)
print(train_data.columns)


6
17
14
12
15
18
Index(['id', 'label', 'statement', 'subject', 'speaker', 'job', 'state',
       'party', 'barely-true', 'false', 'half-true', 'mostly-true',
       'pants-fire', 'venue', 'output', 'speaker_id', 'job_id', 'party_id',
       'state_id', 'subject_id', 'venue_id', 'word_id', 'pos_id', 'dep_id'],
      dtype='object')


In [54]:
import numpy as np
from tensorflow.keras.preprocessing import sequence
from tensorflow import keras

#####################
###  HYPERPARAMS  ###
#####################

vocab_length = len(vocabulary_dict.keys())
hidden_size = EMBED_DIM  # Should be same as EMBED_DIM
lstm_size = 100
num_steps = 15
num_epochs = 30
batch_size = 40

# Hyperparameters for CNN
kernel_sizes = [3, 3, 3]
filter_size = 128

# Meta data related hyperparameters (using max ID + 1 to avoid IndexError)
num_party = train_data['party_id'].max() + 1
num_state = train_data['state_id'].max() + 1
num_venue = train_data['venue_id'].max() + 1
num_job = train_data['job_id'].max() + 1
num_sub = train_data['subject_id'].max() + 1
num_speaker = train_data['speaker_id'].max() + 1

print("Party classes:", num_party)
print("State classes:", num_state)
print("Venue classes:", num_venue)
print("Job classes:", num_job)
print("Subject classes:", num_sub)
print("Speaker classes:", num_speaker)
print("Train columns:", train_data.columns)

#####################
###   DATA PREP   ###
#####################

# Word ID sequences
X_train = sequence.pad_sequences(train_data['word_id'], maxlen=num_steps, padding='post', truncating='post')
X_val = sequence.pad_sequences(val_data['word_id'], maxlen=num_steps, padding='post', truncating='post')
X_test = sequence.pad_sequences(test_data['word_id'], maxlen=num_steps, padding='post', truncating='post')

# Labels (one-hot encoded)
Y_train = keras.utils.to_categorical(train_data['output'], num_classes=6)
Y_val = keras.utils.to_categorical(val_data['output'], num_classes=6)

# POS tag sequences
X_train_pos = sequence.pad_sequences(train_data['pos_id'], maxlen=num_steps, padding='post', truncating='post')
X_val_pos = sequence.pad_sequences(val_data['pos_id'], maxlen=num_steps, padding='post', truncating='post')
X_test_pos = sequence.pad_sequences(test_data['pos_id'], maxlen=num_steps, padding='post', truncating='post')

# DEP tag sequences
X_train_dep = sequence.pad_sequences(train_data['dep_id'], maxlen=num_steps, padding='post', truncating='post')
X_val_dep = sequence.pad_sequences(val_data['dep_id'], maxlen=num_steps, padding='post', truncating='post')
X_test_dep = sequence.pad_sequences(test_data['dep_id'], maxlen=num_steps, padding='post', truncating='post')

# One-hot encoded meta features
party_train = keras.utils.to_categorical(train_data['party_id'], num_classes=num_party)
state_train = keras.utils.to_categorical(train_data['state_id'], num_classes=num_state)
venue_train = keras.utils.to_categorical(train_data['venue_id'], num_classes=num_venue)
job_train = keras.utils.to_categorical(train_data['job_id'], num_classes=num_job)
subject_train = keras.utils.to_categorical(train_data['subject_id'], num_classes=num_sub)
speaker_train = keras.utils.to_categorical(train_data['speaker_id'], num_classes=num_speaker)

X_train_meta = np.hstack((party_train, state_train, venue_train, job_train, subject_train, speaker_train))

# Validation meta
party_val = keras.utils.to_categorical(val_data['party_id'], num_classes=num_party)
state_val = keras.utils.to_categorical(val_data['state_id'], num_classes=num_state)
venue_val = keras.utils.to_categorical(val_data['venue_id'], num_classes=num_venue)
job_val = keras.utils.to_categorical(val_data['job_id'], num_classes=num_job)
subject_val = keras.utils.to_categorical(val_data['subject_id'], num_classes=num_sub)
speaker_val = keras.utils.to_categorical(val_data['speaker_id'], num_classes=num_speaker)

X_val_meta = np.hstack((party_val, state_val, venue_val, job_val, subject_val, speaker_val))

# Test meta
party_test = keras.utils.to_categorical(test_data['party_id'], num_classes=num_party)
state_test = keras.utils.to_categorical(test_data['state_id'], num_classes=num_state)
venue_test = keras.utils.to_categorical(test_data['venue_id'], num_classes=num_venue)
job_test = keras.utils.to_categorical(test_data['job_id'], num_classes=num_job)
subject_test = keras.utils.to_categorical(test_data['subject_id'], num_classes=num_sub)
speaker_test = keras.utils.to_categorical(test_data['speaker_id'], num_classes=num_speaker)

X_test_meta = np.hstack((party_test, state_test, venue_test, job_test, subject_test, speaker_test))


Party classes: 6
State classes: 17
Venue classes: 14
Job classes: 13
Subject classes: 15
Speaker classes: 18
Train columns: Index(['id', 'label', 'statement', 'subject', 'speaker', 'job', 'state',
       'party', 'barely-true', 'false', 'half-true', 'mostly-true',
       'pants-fire', 'venue', 'output', 'speaker_id', 'job_id', 'party_id',
       'state_id', 'subject_id', 'venue_id', 'word_id', 'pos_id', 'dep_id'],
      dtype='object')


In [56]:
print(X_train_meta.shape, X_val_meta.shape, X_test_meta.shape)
print(X_train.shape, X_val.shape, X_test.shape)
print(X_train_pos.shape, X_val_pos.shape, X_test_pos.shape)
print(X_train_dep.shape, X_val_dep.shape, X_test_dep.shape)
print(Y_train.shape, Y_val.shape)

print("X_train_dep:\n", X_train_dep)
print("X_val_dep:\n", X_val_dep)
print("X_test_dep:\n", X_test_dep)


(10240, 83) (1284, 83) (1267, 83)
(10240, 15) (1284, 15) (1267, 15)
(10240, 15) (1284, 15) (1267, 15)
(10240, 15) (1284, 15) (1267, 15)
(10240, 6) (1284, 6)
X_train_dep:
 [[ 6  4 10 ...  2  0  0]
 [10  6  4 ...  5 10 10]
 [ 3  5  6 ... 10  4  8]
 ...
 [ 6  4  5 ... 10  0  9]
 [ 6 10  4 ...  0  0  0]
 [ 4  5  1 ...  8  8  1]]
X_val_dep:
 [[ 5  6  7 ...  0  0  0]
 [10 10 10 ...  3  8  0]
 [ 6 10  5 ... 10  1  4]
 ...
 [ 3  5 10 ...  3  2  0]
 [ 4  7  5 ... 10  3  8]
 [ 4  5  6 ...  4  3  2]]
X_test_dep:
 [[10  4  8 ...  0  0  0]
 [ 5  6  1 ...  0  0  0]
 [ 6  3  5 ...  0  0  0]
 ...
 [ 1  4  7 ...  7  5 10]
 [ 6  4  3 ... 10  5 10]
 [ 6  4  5 ... 10  5  9]]


In [58]:
def train(model, name, use_pos=False, use_meta=False, use_dep=False):
    from keras.callbacks import TensorBoard
    from keras import optimizers

    sgd = optimizers.SGD(learning_rate=0.025, clipvalue=0.3, nesterov=True)
    adam = optimizers.Adam(learning_rate=0.000075)

    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

    tb = TensorBoard()
    csv_logger = keras.callbacks.CSVLogger('training.log')
    filepath = name + "_weights_best.hdf5"
    checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_categorical_accuracy',
                                                 verbose=1, save_best_only=True, mode='max')

    fit_kwargs = {
        'epochs': num_epochs,
        'batch_size': batch_size,
        'callbacks': [tb, csv_logger, checkpoint]
    }

    # Determine input structure
    inputs = {'main_input': X_train}
    val_inputs = {'main_input': X_val}

    if use_pos:
        inputs['pos_input'] = X_train_pos
        val_inputs['pos_input'] = X_val_pos
    if use_meta:
        inputs['aux_input'] = X_train_meta
        val_inputs['aux_input'] = X_val_meta
    if use_dep:
        inputs['dep_input'] = X_train_dep
        val_inputs['dep_input'] = X_val_dep

    model.fit(inputs, {'main_output': Y_train}, validation_data=(val_inputs, {'main_output': Y_val}), **fit_kwargs)


def evaluate(name, use_pos=False, use_meta=False, use_dep=False):
    from keras.models import load_model
    import numpy as np
    import _pickle as cPickle
    import operator

    model1 = load_model(name + '_weights_best.hdf5')

    test_inputs = [X_test]
    if use_pos:
        test_inputs.append(X_test_pos)
    if use_dep:
        test_inputs.append(X_test_dep)
    if use_meta:
        test_inputs.append(X_test_meta)

    preds = model1.predict(test_inputs, batch_size=batch_size, verbose=1)

    false_worst = {}
    true_best = {}
    label_list = ['pants-fire', 'false', 'barely-true', 'half-true', 'mostly-true', 'true']

    Y_test_gt = list(test_data['output'])
    predictions = np.array([np.argmax(pred) for pred in preds])

    for p in range(len(preds)):
        if np.argmax(preds[p]) == 0:
            false_worst[p] = preds[p][0]
        elif np.argmax(preds[p]) == 5:
            true_best[p] = preds[p][5]

    print(len(predictions) == len(Y_test_gt))
    correct = np.sum(predictions == Y_test_gt)
    print("Correctly Predicted : ", correct, "/", len(Y_test_gt))
    print("Accuracy : ", correct * 100.0 / len(Y_test_gt))
    cPickle.dump(predictions, open(name + '_predictions.p', 'wb'))
    return false_worst, true_best


def print_best_false_true_predicted(fw, tb):
    import operator

    sorted_false = sorted(fw.items(), key=operator.itemgetter(1), reverse=True)
    sorted_true = sorted(tb.items(), key=operator.itemgetter(1), reverse=True)

    print('*****************************************************************')
    print('******************** False statements *************************')

    for t in sorted_false[:5]:
        print(t[1])
        print(test_data.loc[t[0]])
        print('=============')

    print('*****************************************************************')
    print('******************** True Statements *************************')

    for t in sorted_true[:5]:
        print(t[1])
        print(test_data.loc[t[0]])
        print('=============')


In [60]:
#####################
###  HYPERPARAMS  ###
#####################
vocab_length = len(vocabulary_dict.keys())
hidden_size = EMBED_DIM  # Has to be same as EMBED_DIM
lstm_size = 100
num_steps = 15
num_epochs = 30
batch_size = 40

# Hyperparams for CNN
kernel_sizes = [3, 3, 3]
filter_size = 128

# Meta data related hyperparams
num_party = len(train_data.party_id.unique())
num_state = len(train_data.state_id.unique())
num_venue = len(train_data.venue_id.unique())
num_job = len(train_data.job_id.unique())
num_sub = len(train_data.subject_id.unique())
num_speaker = len(train_data.speaker_id.unique())

# Print metadata sizes
print("Number of unique parties:", num_party)
print("Number of unique states:", num_state)
print("Number of unique venues:", num_venue)
print("Number of unique jobs:", num_job)
print("Number of unique subjects:", num_sub)
print("Number of unique speakers:", num_speaker)

# Print column names of train_data
print("Train data columns:", train_data.columns.tolist())


Number of unique parties: 6
Number of unique states: 17
Number of unique venues: 14
Number of unique jobs: 12
Number of unique subjects: 15
Number of unique speakers: 18
Train data columns: ['id', 'label', 'statement', 'subject', 'speaker', 'job', 'state', 'party', 'barely-true', 'false', 'half-true', 'mostly-true', 'pants-fire', 'venue', 'output', 'speaker_id', 'job_id', 'party_id', 'state_id', 'subject_id', 'venue_id', 'word_id', 'pos_id', 'dep_id']


In [63]:
import numpy as np
import keras
from keras.models import Model
from keras.layers import Input, Embedding, Conv1D, GlobalMaxPool1D, Dense, Dropout
from keras.optimizers import SGD, Adam
from keras.callbacks import TensorBoard, CSVLogger, ModelCheckpoint

#####################
###  HYPERPARAMS  ###
#####################
vocab_length = len(vocabulary_dict.keys())
EMBED_DIM = 100
hidden_size = EMBED_DIM  # Has to be same as EMBED_DIM
lstm_size = 100
num_steps = 15
num_epochs = 30
batch_size = 40

# Hyperparams for CNN
kernel_sizes = [3, 3, 3]
filter_size = 128

# Meta data related hyper params
num_party = len(train_data.party_id.unique())
num_state = len(train_data.state_id.unique())
num_venue = len(train_data.venue_id.unique())
num_job = len(train_data.job_id.unique())
num_sub = len(train_data.subject_id.unique())
num_speaker = len(train_data.speaker_id.unique())

print(num_party)
print(num_state)
print(num_venue)
print(num_job)
print(num_sub)
print(num_speaker)
print(train_data.columns)

######################
###   MODEL CODE   ###
######################
kernel_stmt = []
kernel_pos = []
kernel_dep = []

use_pos = False
use_meta = True
use_dep = True

# Statement input
statement_input = Input(shape=(num_steps,), dtype='int32', name='main_input')
x_stmt = Embedding(vocab_length + 1, EMBED_DIM, weights=[embedding_matrix], input_length=num_steps, trainable=False)(statement_input)

# POS input
pos_input = Input(shape=(num_steps,), dtype='int32', name='pos_input')
vocab_size_pos = max(pos_dict.values()) + 1  # Total unique POS tags + padding token
embedding_dim_pos = pos_embeddings.shape[1]  # Embedding dimension for POS
x_pos = Embedding(vocab_size_pos, embedding_dim_pos, weights=[pos_embeddings], input_length=num_steps, trainable=False)(pos_input)

# Dependency input
dep_input = Input(shape=(num_steps,), dtype='int32', name='dep_input')
vocab_size_dep = max(dep_dict.values()) + 1  # Total unique dependency tags + padding token
embedding_dim_dep = dep_embeddings.shape[1]  # Embedding dimension for dependencies
x_dep = Embedding(vocab_size_dep, embedding_dim_dep, weights=[dep_embeddings], input_length=num_steps, trainable=False)(dep_input)

# Convolutional layers for statement, POS, and dependency inputs
for kernel in kernel_sizes:
    x_1 = Conv1D(filters=filter_size, kernel_size=kernel)(x_stmt)
    x_1 = GlobalMaxPool1D()(x_1)
    kernel_stmt.append(x_1)

    x_2 = Conv1D(filters=filter_size, kernel_size=kernel)(x_pos)
    x_2 = GlobalMaxPool1D()(x_2)
    kernel_pos.append(x_2)

    x_3 = Conv1D(filters=filter_size, kernel_size=kernel)(x_dep)
    x_3 = GlobalMaxPool1D()(x_3)
    kernel_dep.append(x_3)

# Concatenate outputs of Conv1D layers
conv_in1 = keras.layers.concatenate(kernel_stmt)
conv_in1 = Dropout(0.6)(conv_in1)
conv_in1 = Dense(128, activation='relu')(conv_in1)

conv_in2 = keras.layers.concatenate(kernel_pos)
conv_in2 = Dropout(0.6)(conv_in2)
conv_in2 = Dense(128, activation='relu')(conv_in2)

conv_in3 = keras.layers.concatenate(kernel_dep)
conv_in3 = Dropout(0.6)(conv_in3)
conv_in3 = Dense(128, activation='relu')(conv_in3)

# Meta data input
meta_input = Input(shape=(X_train_meta.shape[1],), name='aux_input')
x_meta = Dense(64, activation='relu')(meta_input)

# Concatenate layers based on user inputs
if use_pos and use_meta:
    if use_dep:
        x = keras.layers.concatenate([conv_in1, conv_in2, conv_in3, x_meta])
    else:
        x = keras.layers.concatenate([conv_in1, conv_in2, x_meta])
elif use_meta:
    if use_dep:
        x = keras.layers.concatenate([conv_in1, conv_in3, x_meta])
    else:
        x = keras.layers.concatenate([conv_in1, x_meta])
elif use_pos:
    if use_dep:
        x = keras.layers.concatenate([conv_in1, conv_in2, conv_in3])
    else:
        x = keras.layers.concatenate([conv_in1, conv_in2])
else:
    if use_dep:
        x = keras.layers.concatenate([conv_in1, conv_in3])
    else:
        x = conv_in1

# Output layer
main_output = Dense(6, activation='softmax', name='main_output')(x)

# Model compilation
if use_pos and use_meta:
    if use_dep:
        model_cnn = Model(inputs=[statement_input, pos_input, dep_input, meta_input], outputs=[main_output])
    else:
        model_cnn = Model(inputs=[statement_input, pos_input, meta_input], outputs=[main_output])
elif use_meta:
    if use_dep:
        model_cnn = Model(inputs=[statement_input, dep_input, meta_input], outputs=[main_output])
    else:
        model_cnn = Model(inputs=[statement_input, meta_input], outputs=[main_output])
elif use_pos:
    if use_dep:
        model_cnn = Model(inputs=[statement_input, pos_input, dep_input], outputs=[main_output])
    else:
        model_cnn = Model(inputs=[statement_input, pos_input], outputs=[main_output])
else:
    if use_dep:
        model_cnn = Model(inputs=[statement_input, dep_input], outputs=[main_output])
    else:
        model_cnn = Model(inputs=[statement_input], outputs=[main_output])

# Optimizers and compilation
sgd = SGD(lr=0.025, clipvalue=0.3, nesterov=True)
adam = Adam(lr=0.000075, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model_cnn.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

# Callbacks
tb = TensorBoard()
csv_logger = CSVLogger('training.log')
filepath = "model_weights_best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_categorical_accuracy', verbose=1, save_best_only=True, mode='max')

# Train the model
model_cnn.fit(
    {'main_input': X_train, 'pos_input': X_train_pos, 'dep_input': X_train_dep, 'aux_input': X_train_meta},
    {'main_output': Y_train},
    epochs=num_epochs, batch_size=batch_size,
    validation_data=(
        {'main_input': X_val, 'pos_input': X_val_pos, 'dep_input': X_val_dep, 'aux_input': X_val_meta},
        {'main_output': Y_val}
    ),
    callbacks=[tb, csv_logger, checkpoint]
)


6
17
14
12
15
18
Index(['id', 'label', 'statement', 'subject', 'speaker', 'job', 'state',
       'party', 'barely-true', 'false', 'half-true', 'mostly-true',
       'pants-fire', 'venue', 'output', 'speaker_id', 'job_id', 'party_id',
       'state_id', 'subject_id', 'venue_id', 'word_id', 'pos_id', 'dep_id'],
      dtype='object')


ValueError: Argument(s) not recognized: {'lr': 0.025}

In [65]:
#plot_model(model, to_file='model_lstm.png', show_shapes=True, show_layer_names=True)
#SVG(model_to_dot(model,show_shapes=True).create(prog='dot', format='svg'))
print(model_cnn.summary())


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ main_input          │ (None, 15)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dep_input           │ (None, 15)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, 15, 100)   │  1,240,900 │ main_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_4         │ (None, 15, 11)    │        121 │ dep_input[0][0]   │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 13, 128)   │     38,528 │ embedding_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_3 (Conv1D)   │ (None, 13, 128)   │     38,528 │ embedding_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_6 (Conv1D)   │ (None, 13, 128)   │     38,528 │ embedding_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 13, 128)   │      4,352 │ embedding_4[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_5 (Conv1D)   │ (None, 13, 128)   │      4,352 │ embedding_4[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_8 (Conv1D)   │ (None, 13, 128)   │      4,352 │ embedding_4[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 128)       │          0 │ conv1d[0][0]      │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 128)       │          0 │ conv1d_3[0][0]    │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 128)       │          0 │ conv1d_6[0][0]    │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 128)       │          0 │ conv1d_2[0][0]    │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 128)       │          0 │ conv1d_5[0][0]    │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 128)       │          0 │ conv1d_8[0][0]    │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 384)       │          0 │ global_max_pooli… │
│ (Concatenate)       │                   │            │ global_max_pooli… │
│                     │                   │            │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 384)       │          0 │ global_max_pooli… │
│ (Concatenate)       │                   │            │ global_max_pooli… │
│                     │                   │            │ global_max_pooli

 Total params: 1,475,523 (5.63 MB)

 Trainable params: 234,502 (916.02 KB)

 Non-trainable params: 1,241,021 (4.73 MB)

None


In [79]:
from keras.utils import to_categorical
import numpy as np

# Check if Y_train and Y_val are already one-hot encoded
if len(Y_train.shape) == 2 and Y_train.shape[1] == 6:  # Assuming 6 classes
    print("Labels are already one-hot encoded.")
    Y_train_one_hot = Y_train.astype(np.float32)  # Ensure dtype is float32
    Y_val_one_hot = Y_val.astype(np.float32)  # Ensure dtype is float32
else:
    # If they are integer labels, use to_categorical to convert to one-hot encoding
    print("Converting integer labels to one-hot encoding.")
    Y_train_one_hot = to_categorical(Y_train, num_classes=6).astype(np.float32)
    Y_val_one_hot = to_categorical(Y_val, num_classes=6).astype(np.float32)

# Ensure that the labels are in the right shape (n_samples, num_classes)
print(f"Y_train_one_hot shape: {Y_train_one_hot.shape}")
print(f"Y_val_one_hot shape: {Y_val_one_hot.shape}")

# Now you can train the model using Y_train_one_hot and Y_val_one_hot
model_cnn.fit(
    {'main_input': X_train, 'pos_input': X_train_pos, 'aux_input': X_train_meta, 'dep_input': X_train_dep},
    {'main_output': Y_train_one_hot},  # Use one-hot encoded labels
    epochs=num_epochs, batch_size=batch_size,
    validation_data=(
        {'main_input': X_val, 'pos_input': X_val_pos, 'aux_input': X_val_meta, 'dep_input': X_val_dep},
        {'main_output': Y_val_one_hot}  # Use one-hot encoded labels
    ),
    callbacks=[tb, csv_logger, checkpoint]
)


Converting integer labels to one-hot encoding.
Y_train_one_hot shape: (10240, 6)
Y_val_one_hot shape: (1284, 6)
Epoch 1/30


TypeError: Expected float32, but got main_output of type 'str'.

In [ ]:
(fw, tb) = evaluate('cnn', use_pos, use_meta, use_dep)
print_best_false_true_predicted(fw, tb)


1267/1267 [==============================] - 1s 744us/step
True
Correctly Predicted :  341 / 1267
Accuracy :  26.91397000789266
*****************************************************************
******************** False statements *************************
0.7503765
id             9051.json                                                                                                                                                                                                                    
label          pants-fire                                                                                                                                                                                                                   
statement      Says President Barack Obama told a room of students, Children, every time I clap my hands together, a child in America dies from gun violence, and then a child told him he could solve the problem by not clapping any more.
subject        guns  

In [ ]:
#####################
###  HYPERPARAMS  ###
#####################
vocab_length = len(vocabulary_dict.keys())
hidden_size = EMBED_DIM #Has to be same as EMBED_DIM
lstm_size = 100
num_steps = 15
num_epochs = 30
batch_size = 40

#Hyperparams for CNN
kernel_sizes = [3,3,3]
filter_size = 128

#Meta data related hyper params
num_party = len(train_data.party_id.unique())
num_state = len(train_data.state_id.unique())
num_venue = len(train_data.venue_id.unique())
num_job = len(train_data.job_id.unique())
num_sub = len(train_data.subject_id.unique())
num_speaker = len(train_data.speaker_id.unique())
print num_party
print num_state
print num_venue
print num_job
print num_sub
print num_speaker
print train_data.columns

6
17
14
13
15
18
Index([u'id', u'label', u'statement', u'subject', u'speaker', u'job', u'state',
       u'party', u'barely-true', u'false', u'half-true', u'mostly-true',
       u'pants-fire', u'venue', u'output', u'speaker_id', u'job_id',
       u'party_id', u'state_id', u'subject_id', u'venue_id', u'word_id',
       u'pos_id', u'dep_id'],
      dtype='object')


In [ ]:
use_pos=False
use_meta=True
use_dep=True

# LSTM model
model_lstm = Sequential()
model_lstm.add(Embedding(vocab_length+1, hidden_size, input_length=num_steps))
model_lstm.add(Bidirectional(LSTM(hidden_size)))
model_lstm.add(Dense(6, activation='softmax'))


# statement embed LSTM
statement_input = Input(shape=(num_steps,), dtype='int32', name='main_input')
x = Embedding(vocab_length+1,EMBED_DIM,weights=[embedding_matrix],input_length=num_steps,trainable=False)(statement_input)
lstm_in = LSTM(lstm_size,dropout=0.2)(x)



# pos embed LSTM
pos_input = Input(shape=(num_steps,), dtype='int32', name='pos_input')
x2 = Embedding(max(pos_dict.values()), max(pos_dict.values()), weights=[pos_embeddings], input_length=num_steps, trainable=False)(pos_input)
lstm_in2 = LSTM(lstm_size, dropout=0.2)(x2)


# dep embed LSTM
dep_input = Input(shape=(num_steps,), dtype='int32', name='dep_input')
x3 = Embedding(max(dep_dict.values()), max(dep_dict.values()), weights=[dep_embeddings], input_length=num_steps, trainable=False)(dep_input)
lstm_in3 = LSTM(lstm_size, dropout=0.2)(x3)


# meta data Dense
meta_input = Input(shape=(X_train_meta.shape[1],), name='aux_input')
x_meta = Dense(64, activation='relu')(meta_input)


if use_pos and use_meta:
  if use_dep:
    x = keras.layers.concatenate([lstm_in, lstm_in2, lstm_in3, x_meta])
  else:
    x = keras.layers.concatenate([lstm_in, lstm_in2, x_meta])
elif use_meta:
  if use_dep:
    x = keras.layers.concatenate([lstm_in, lstm_in3, x_meta])
  else:
    x = keras.layers.concatenate([lstm_in, x_meta])
elif use_pos:
  if use_dep:
    x = keras.layers.concatenate([lstm_in, lstm_in3, lstm_in2])
  else:
    x = keras.layers.concatenate([lstm_in, lstm_in2])
else:
  if use_dep:
    x = keras.layers.concatenate([lstm_in, lstm_in3])
  else:
    x = lstm_in




main_output = Dense(6, activation='softmax', name='main_output')(x)

if use_pos and use_meta:
  if use_dep:
    model_lstm = Model(inputs=[statement_input, pos_input, dep_input, meta_input], outputs=[main_output])
  else:
    model_lstm = Model(inputs=[statement_input, pos_input, meta_input], outputs=[main_output])
elif use_meta:
  if use_dep:
    model_lstm = Model(inputs=[statement_input, dep_input, meta_input], outputs=[main_output])
  else:
    model_lstm = Model(inputs=[statement_input, meta_input], outputs=[main_output])
elif use_pos:
  if use_dep:
    model_lstm = Model(inputs=[statement_input, dep_input, pos_input], outputs=[main_output])
  else:
    model_lstm = Model(inputs=[statement_input, pos_input], outputs=[main_output])
else:
  if use_dep:
    model_lstm = Model(inputs=[statement_input, dep_input], outputs=[main_output])
  else:
    model_lstm = Model(inputs=[statement_input], outputs=[main_output])

In [ ]:
print model_lstm.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, 15)           0                                            
__________________________________________________________________________________________________
dep_input (InputLayer)          (None, 15)           0                                            
__________________________________________________________________________________________________
embedding_51 (Embedding)        (None, 15, 100)      1240900     main_input[0][0]                 
__________________________________________________________________________________________________
embedding_53 (Embedding)        (None, 15, 10)       100         dep_input[0][0]                  
__________________________________________________________________________________________________
aux_input 

In [ ]:
train(model_lstm,'lstm', use_pos, use_meta, use_dep)


Train on 10240 samples, validate on 1284 samples
Epoch 1/30
10240/10240 [==============================] - 49s 5ms/step - loss: 1.7592 - categorical_accuracy: 0.2089 - val_loss: 1.7502 - val_categorical_accuracy: 0.2329

Epoch 00001: val_categorical_accuracy improved from -inf to 0.23287, saving model to lstm_weights_best.hdf5
Epoch 2/30
10240/10240 [==============================] - 44s 4ms/step - loss: 1.7430 - categorical_accuracy: 0.2242 - val_loss: 1.7399 - val_categorical_accuracy: 0.2336

Epoch 00002: val_categorical_accuracy improved from 0.23287 to 0.23364, saving model to lstm_weights_best.hdf5
Epoch 3/30
10240/10240 [==============================] - 44s 4ms/step - loss: 1.7320 - categorical_accuracy: 0.2374 - val_loss: 1.7330 - val_categorical_accuracy: 0.2321

Epoch 00003: val_categorical_accuracy did not improve from 0.23364
Epoch 4/30
10240/10240 [==============================] - 45s 4ms/step - loss: 1.7225 - categorical_accuracy: 0.2460 - val_loss: 1.7194 - val_categor

In [ ]:
(fw,tb) = evaluate('lstm', use_pos, use_meta, use_dep)
print_best_false_true_predicted(fw, tb)

1267/1267 [==============================] - 5s 4ms/step
True
Correctly Predicted :  360 / 1267
Accuracy :  28.41357537490134
*****************************************************************
******************** False statements *************************
0.7088227
id             9051.json                                                                                                                                                                                                                    
label          pants-fire                                                                                                                                                                                                                   
statement      Says President Barack Obama told a room of students, Children, every time I clap my hands together, a child in America dies from gun violence, and then a child told him he could solve the problem by not clapping any more.
subject        guns    